In [1]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="quick-starts-ws-266842")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-266842
Azure region: eastus2
Subscription id: 9a7511b8-150f-4a58-8528-3e7d50216c31
Resource group: aml-quickstarts-266842


In [2]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.exceptions import ComputeTargetException

cluster_name = "assignment1"

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

try:
    compute_target = ComputeTarget(workspace=ws, name=cluster_name)
except ComputeTargetException as e:
    print(e)
    compute_config = AmlCompute.provisioning_configuration(vm_size="STANDARD_D2_V2", max_nodes=4)
    compute_target = ComputeTarget.create(ws,cluster_name,compute_config)

compute_target.wait_for_completion(show_output=True)

Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [4]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.core import Environment, ScriptRunConfig
import os

# Specify parameter sampler
ps = RandomParameterSampling({
    '--C' : uniform(0.5, 1.5),
    '--max_iter': choice(50,100,150)
})

# Specify a Policy
policy = BanditPolicy(evaluation_interval=2,slack_factor=0.1)

if "training" not in os.listdir():
    os.mkdir("./training")

# Setup environment for your training run
sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='conda_dependencies.yml')

# Create a ScriptRunConfig Object to specify the configuration details of your training job
src = ScriptRunConfig(
    source_directory=".",
    script="train.py",
    arguments=None,
    compute_target=compute_target,
    environment=sklearn_env
)

# Create a HyperDriveConfig using the src object, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(
    run_config=src,
    policy=policy,
    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
    primary_metric_name="Accuracy",
    max_total_runs=20,
    max_concurrent_runs=4,
    hyperparameter_sampling=ps
)

In [5]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

hyperdrive_run = exp.submit(config=hyperdrive_config)

RunDetails(hyperdrive_run).show()

2024-08-20 03:39:57.334131: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /anaconda/envs/azureml_py38/lib/python3.9/site-packages/cv2/../../lib64:
2024-08-20 03:39:57.334189: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
Failed to load image Python extension: libc10_cuda.so: cannot open shared object file: No such file or directory


_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [6]:
import joblib
# Get your best run and save the model from that run.

hyperdrive_run.wait_for_completion(show_output=True)

assert(hyperdrive_run.get_status=="Completed")



RunId: HD_cce6fcf1-6ff9-4ecc-ac16-73cf48bb6896
Web View: https://ml.azure.com/runs/HD_cce6fcf1-6ff9-4ecc-ac16-73cf48bb6896?wsid=/subscriptions/9a7511b8-150f-4a58-8528-3e7d50216c31/resourcegroups/aml-quickstarts-266842/workspaces/quick-starts-ws-266842&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254

Streaming azureml-logs/hyperdrive.txt

[2024-08-20T03:38:55.069312][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space
[2024-08-20T03:38:55.5244679Z][SCHEDULER][INFO]Scheduling job, id='HD_cce6fcf1-6ff9-4ecc-ac16-73cf48bb6896_0' 
[2024-08-20T03:38:55.7824878Z][SCHEDULER][INFO]Scheduling job, id='HD_cce6fcf1-6ff9-4ecc-ac16-73cf48bb6896_2' 
[2024-08-20T03:38:55.6452285Z][SCHEDULER][INFO]Scheduling job, id='HD_cce6fcf1-6ff9-4ecc-ac16-73cf48bb6896_1' 
[2024-08-20T03:38:55.8944562Z][SCHEDULER][INFO]Scheduling job, id='HD_cce6fcf1-6ff9-4ecc-ac16-73cf48bb6896_3' 
[2024-08-20T03:38:55.844767][GENERATOR][INFO]Successfully sampled '4' jobs, they will soon be submitted to the execution t

AssertionError: 

In [7]:
hyperdrive_run.get_status

<bound method Run.get_status of Run(Experiment: quick-starts-ws-266842,
Id: HD_cce6fcf1-6ff9-4ecc-ac16-73cf48bb6896,
Type: hyperdrive,
Status: Completed)>

In [8]:
best_run = hyperdrive_run.get_best_run_by_primary_metric()

best_metrics = best_run.get_metrics()
print(best_metrics)

joblib.dump(best_metrics, "./training/hyperdrive_metrics.json")

{'Regularization Strength:': 0.6290233439473092, 'Max iterations:': 150, 'Accuracy': 0.9119878603945372}


['./training/hyperdrive_metrics.json']

In [9]:


from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

tablular_dataset_factory = ["https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"]

dataset = TabularDatasetFactory.from_delimited_files(path=tablular_dataset_factory)

In [11]:
from train import clean_data
import pandas as pd

from azureml.core.dataset import Dataset
import os


# Use the clean_data function to clean your data.
x, y = clean_data(dataset)

train_data = x.join(y)
print(train_data.head())

if not os.path.isdir("data"):
    os.mkdir("data")

pd.DataFrame(train_data).to_csv("data/train_data.csv", index=False)

datastore = ws.get_default_datastore()

datastore.upload(
    src_dir="./data", target_path="bankmarketing", overwrite=True, show_progress=True
)

train_ds = Dataset.Tabular.from_delimited_files(path=datastore.path("bankmarketing/train_data.csv"))

{'infer_column_types': 'False', 'activity': 'to_pandas_dataframe'}
{'infer_column_types': 'False', 'activity': 'to_pandas_dataframe', 'activityApp': 'TabularDataset'}
   age  marital  default  housing  loan  month  day_of_week  duration  \
0   57        1        0        0     1      5            1       371   
1   55        1        0        1     0      5            4       285   
2   33        1        0        0     0      5            5        52   
3   36        1        0        0     0      6            5       355   
4   27        1        0        1     0      7            5       189   

   campaign  pdays  ...  contact_telephone  education_basic.4y  \
0         1    999  ...                  0                   0   
1         2    999  ...                  1                   0   
2         1    999  ...                  0                   0   
3         4    999  ...                  1                   0   
4         2    999  ...                  0                   0  

"Datastore.upload" is deprecated after version 1.0.69. Please use "Dataset.File.upload_directory" to upload your files             from a local directory and create FileDataset in single method call. See Dataset API change notice at https://aka.ms/dataset-deprecation.


In [13]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task="classification",
    primary_metric="AUC_weighted",
    compute_target=compute_target,
    training_data=train_ds,
    blocked_models=["KNN", "LinearSVM"],
    label_column_name="y",
    max_concurrent_iterations=4,
    enable_onnx_compatible_models=True,
    n_cross_validations=5)

In [14]:
# Submit your automl run

auto_run = exp.submit(config=automl_config)

RunDetails(auto_run).show()

Submitting remote run.


Experiment,Id,Type,Status,Details Page,Docs Page
quick-starts-ws-266842,AutoML_aa8b3bb9-3540-4f1f-82b8-f06b2bdca81b,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation


_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

Current provisioning state of AmlCompute is "Deleting"



In [15]:
# Retrieve and save your best automl model.

auto_run.wait_for_completion(show_output=True)

assert(auto_run.get_status() == "Completed")

best_auto_run, autorun_final_model = auto_run.get_output()

joblib.dump(autorun_final_model, "./training/automl_model.pkl")

Experiment,Id,Type,Status,Details Page,Docs Page
quick-starts-ws-266842,AutoML_aa8b3bb9-3540-4f1f-82b8-f06b2bdca81b,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation



Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

********************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because the input data has bias towards one class.
+------------------------------+--------------------------------+--------------------------------------+
|Size of the smallest class    |Name/Label of the smallest class|Nu

2024-08-20 04:49:32.022426: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuda.so.1
2024-08-20 04:49:38.137408: E tensorflow/stream_executor/cuda/cuda_driver.cc:328] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2024-08-20 04:49:38.137530: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (assignment1compute): /proc/driver/nvidia/version does not exist


['./training/automl_model.pkl']

In [17]:
print(compute_target.get())

compute_target.delete()

{'id': '/subscriptions/9a7511b8-150f-4a58-8528-3e7d50216c31/resourceGroups/aml-quickstarts-266842/providers/Microsoft.MachineLearningServices/workspaces/quick-starts-ws-266842/computes/assignment1', 'name': 'assignment1', 'type': 'Microsoft.MachineLearningServices/workspaces/computes', 'location': 'eastus2', 'tags': {'task': 'assignment1'}, 'properties': {'createdOn': '2024-08-20T03:22:36.5279838+00:00', 'modifiedOn': '2024-08-20T03:22:46.442256+00:00', 'disableLocalAuth': True, 'description': None, 'resourceId': None, 'computeType': 'AmlCompute', 'computeLocation': 'eastus2', 'provisioningState': 'Succeeded', 'provisioningErrors': None, 'provisioningWarnings': {}, 'isAttachedCompute': False, 'properties': {'vmSize': 'Standard_D2_v2', 'vmPriority': 'Dedicated', 'scaleSettings': {'maxNodeCount': 4, 'minNodeCount': 0, 'nodeIdleTimeBeforeScaleDown': 'PT2M'}, 'subnet': None, 'currentNodeCount': 0, 'targetNodeCount': 0, 'nodeStateCounts': {'preparingNodeCount': 0, 'runningNodeCount': 0, 'id

In [21]:
print(compute_target.get())


None
